In [66]:
from glob import glob
import pandas as pd
import numpy as np
import os
import json
from collections import defaultdict

In [69]:
# df_movies = pd.read_csv('dataset/tmdb_5000_movies.csv')
# df_movies = df_movies.dropna(subset=['overview', 'release_date', 'runtime'])
# df_movies.isnull().sum()
# df_movies.to_csv("dataset/tmdb_5000_movies_nonull.csv")

In [75]:
df = pd.read_csv('dataset/tmdb_5000_movies_nonull.csv')
# df_movies.loc[:,['id', 'keywords', 'genres', 'title']]
# df_credits = pd.read_csv('dataset/tmdb_5000_credits.csv')

df = df[['budget', 'genres', 'keywords', 'original_language'
                      , 'overview', 'popularity', 'production_companies'
                     , 'production_countries', 'release_date', 'revenue'
                     , 'runtime', 'spoken_languages', 'tagline', 'title'
                     , 'vote_average', 'vote_count']]
display(df.sort_values(by='release_date', ascending=False))
# display(df_credits.head())

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count
4253,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...",[],en,"In 1979, an Indian family moves to America wit...",0.710870,[],[],2017-02-03,0,102.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",It’s better to stand out than to fit in.,Growing Up Smith,7.4,7
3407,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...","[{""id"": 1415, ""name"": ""small town""}, {""id"": 46...",en,Set in a small town near the North Pole where ...,3.139294,[],[],2016-10-02,0,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",NaN,Two Lovers and a Bear,6.8,27
3301,8000000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 1650, ""name"": ""cook""}, {""id"": 6054, ""n...",en,A unique friendship develops when a little gir...,7.828459,"[{""name"": ""Envision Media Arts"", ""id"": 19456},...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-16,0,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",He was the one person she could always count on.,Mr. Church,7.0,129
3248,0,"[{""id"": 12, ""name"": ""Adventure""}]","[{""id"": 3405, ""name"": ""blow job""}, {""id"": 1173...",en,When his hard-earned kicks get snatched by a l...,3.467923,"[{""name"": ""Bystorm Films"", ""id"": 2903}, {""name...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-09,0,80.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",They aren't just shoes,Kicks,7.5,18
4716,8500000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 2831, ""name"": ""slavery""}]",en,"Nat Turner, a former slave in America, leads a...",9.452808,"[{""name"": ""Phantom Four"", ""id"": 423}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-09,15861566,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Untold Story of Nat Turner,The Birth of a Nation,6.5,178
4035,35000000,"[{""id"": 27, ""name"": ""Horror""}]","[{""id"": 726, ""name"": ""drug abuse""}, {""id"": 170...",en,In a desolate community full of drug-addled Ma...,3.674294,"[{""name"": ""Hideaway Pictures"", ""id"": 20068}, {...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-09-02,0,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",NaN,Antibirth,4.8,40
2273,20000000,"[{""id"": 18, ""name"": ""Drama""}]",[],en,The legendary Roberto Duran and his equally le...,7.444189,"[{""name"": ""Weinstein Company, The"", ""id"": 7289...","[{""iso_3166_1"": ""PA"", ""name"": ""Panama""}, {""iso...",2016-08-26,0,105.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",No más. No surrender.,Hands of Stone,6.1,109
357,100000000,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 10085, ""name"": ""betrayal""}, {""id"": 232...",en,A falsely accused nobleman survives years of s...,29.608322,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-08-17,94061311,125.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",First to finish. Last to die.,Ben-Hur,5.3,621
3012,65000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...","[{""id"": 5650, ""name"": ""feral child""}, {""id"": 9...",en,Pete is a mysterious 10-year-old with no famil...,31.974359,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-08-10,143695338,102.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Some secrets are too big to keep.,Pete's Dragon,6.2,679
72,175000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 1296...",en,"From DC Comics comes the Suicide Squad, an ant...",90.237920,"[{""name"": ""DC Comics"", ""id"": 429}, {""name"": ""D...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-08-02,745000000,123.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}

In [77]:
def convert_list(cell):
    """convert the json format to a list of categories"""
    kw_list = []
    for kw in json.loads(cell):
        kw_list.append(kw['name'])
    return kw_list


def larger_n(col, n):
    """filter the column"""
    keywords = defaultdict(int)
    for row in df_movies[col]:
        row = json.loads(row)
        for entry in row:
            keywords[entry['name']] += 1
    kw_cnt = sorted(keywords.items(), key=lambda x: -x[1])
    return [kw[0] for kw in kw_cnt if kw[1] >= n]

df_movies = df

larger_kw = larger_n('keywords', 30)
larger_genre = larger_n('genres', 1000)
larger_pc = larger_n('production_companies', 5)
larger_pcount = larger_n('production_countries', 3)

df_movies['keywords'] = df_movies['keywords'].apply(
    convert_list).apply(lambda cell: [kw for kw in cell if kw in larger_kw])
# df_movies['keywords'] = df_movies['keywords'].apply(convert_list)
df_movies['genres'] = df_movies['genres'].apply(convert_list).apply(
    lambda cell: [kw for kw in cell if kw in larger_genre])
df_movies['production_companies'] = df_movies['production_companies'].apply(convert_list).apply(
    lambda cell: [kw for kw in cell if kw in larger_pc])
df_movies['production_countries'] = df_movies['production_countries'].apply(convert_list).apply(
    lambda cell: [kw for kw in cell if kw in larger_pcount])

df_movies

TypeError: the JSON object must be str, bytes or bytearray, not 'list'

In [71]:
df_movies.production_companies

0       [{"name": "Ingenious Film Partners", "id": 289...
1       [{"name": "Walt Disney Pictures", "id": 2}, {"...
2       [{"name": "Columbia Pictures", "id": 5}, {"nam...
3       [{"name": "Legendary Pictures", "id": 923}, {"...
4             [{"name": "Walt Disney Pictures", "id": 2}]
5       [{"name": "Columbia Pictures", "id": 5}, {"nam...
6       [{"name": "Walt Disney Pictures", "id": 2}, {"...
7       [{"name": "Marvel Studios", "id": 420}, {"name...
8       [{"name": "Warner Bros.", "id": 6194}, {"name"...
9       [{"name": "DC Comics", "id": 429}, {"name": "A...
10      [{"name": "DC Comics", "id": 429}, {"name": "L...
11              [{"name": "Eon Productions", "id": 7576}]
12      [{"name": "Walt Disney Pictures", "id": 2}, {"...
13      [{"name": "Walt Disney Pictures", "id": 2}, {"...
14      [{"name": "Legendary Pictures", "id": 923}, {"...
15      [{"name": "Walt Disney", "id": 5888}, {"name":...
16      [{"name": "Paramount Pictures", "id": 4}, {"na...
17      [{"nam

In [42]:
df_movies = df_movies.dropna(subset=['overview'])

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words='english',binary=True, ngram_range=(1, 2),min_df=0.001)
x = v.fit_transform(df_movies['overview'])

train_features = pd.DataFrame(x.toarray(), columns=v.get_feature_names())


## get dummies

In [44]:
df_movies = df_movies.reset_index(drop=True)

In [53]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
genres_dummies = pd.DataFrame(mlb.fit_transform(
    df_movies['genres']), columns=mlb.classes_).add_prefix('genre_')
keywords_dummies = pd.DataFrame(mlb.fit_transform(
    df_movies['keywords']), columns=mlb.classes_).add_prefix('kw_')
df_new = pd.concat([
    #df_movies.loc[:, ['budget']],
    genres_dummies, keywords_dummies,
    train_features], axis=1)

df_y = df_new.loc[:, df_new.columns.str.startswith('genre_')]
df_X = df_new.drop(
    df_new.columns[df_new.columns.str.startswith('genre_')], inplace=False, axis=1)

In [54]:
df_X

,kw_1970s,kw_3d,kw_adultery,kw_aftercreditsstinger,kw_airplane,kw_alcohol,kw_alien,kw_army,kw_assassin,kw_based on comic book,...,youngest,youth,zealand,zebra,zeus,zoe,zombie,zombies,zone,zoo
0,0,1,0,0,0,0,1,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0,0,0,1,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0,1,0,0,0,0,1,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,0,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,0,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,0,1,0,0,0,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
8,0,0,0,0,0,0,0,0,0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9,0,0,0,0,0,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [55]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, test_size=0.33, random_state=42)

In [56]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier

classifier = BinaryRelevance(RandomForestClassifier())
classifier.fit(train_X, train_y)
predictions = classifier.predict(test_X)

from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.3181818181818182

In [60]:
predictions_dense = predictions.todense()

for i in range(predictions.shape[1]):
    print(accuracy_score(np.asarray(test_y)[:, i], np.asarray(predictions_dense)[:, i]))

0.8188131313131313
0.7083333333333334
0.6843434343434344
0.7689393939393939
